In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df_dirty = pd.read_csv('../data/train.csv')
hdb_detail = pd.read_csv("../data/auxiliary-data/sg-hdb-block-details.csv")
mrt_stations = pd.read_csv("../data/auxiliary-data/sg-mrt-stations.csv")
hawkers = pd.read_csv("../data/auxiliary-data/sg-gov-hawkers.csv")
shopping_malls = pd.read_csv("../data/auxiliary-data/sg-shopping-malls.csv")

In [11]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6_371_000
    dlat = np.deg2rad(lat2 - lat1)
    dlon = np.deg2rad(lon2 - lon1)
    a = np.sin(dlat/2)**2 + np.cos(np.deg2rad(lat1))*np.cos(np.deg2rad(lat2))*np.sin(dlon/2)**2
    return 2*R*np.arcsin(np.sqrt(a))  # meters

def nearest_dist(lat, lon, facility_xy):
    facility_xy = facility_xy.to_numpy()
    dists = haversine(lat, lon, facility_xy[:,0], facility_xy[:,1])
    j = np.argmin(dists)
    return dists[j] # mrt.iloc[j]['NAME']

def classify_floor_range(floor_range):
    # take lower bound
    low = int(floor_range.split(' to ')[0])
    if low <= 6:
        return 'Low'
    elif low <= 12:
        return 'Mid'
    else:
        return 'High'


In [12]:
def add_additional_info(df, hdb_detail, mrt_stations, hawkers, shopping_malls):
    hdb_renamed = hdb_detail.rename(columns={'ADDRESS':'STREET'})

    # get latitude and longtitude
    df = df.merge(
        hdb_renamed[['TOWN','BLOCK','STREET','LATITUDE','LONGITUDE']],
        on=['TOWN','BLOCK','STREET'],
        how='left'
    )

    # get nearest mrt dist (mrt station)
    mrt_xy = mrt_stations[['LATITUDE','LONGITUDE']]
    out = df.apply(lambda r: pd.Series(nearest_dist(r['LATITUDE'], r['LONGITUDE'], mrt_xy),
                    index=['DIST_TO_NEAREST_MRT_M']), axis=1) # in meters
    df = pd.concat([df, out], axis=1)

    # get nearest hawker dist
    hawker_xy = hawkers[['LATITUDE','LONGITUDE']]
    out = df.apply(lambda r: pd.Series(nearest_dist(r['LATITUDE'], r['LONGITUDE'], hawker_xy),
                    index=['DIST_TO_NEAREST_HAWKER_M']), axis=1) # in meters
    df = pd.concat([df, out], axis=1)

    # get nearest shopping mall dist
    shopping_malls_xy = shopping_malls[['LATITUDE','LONGITUDE']]
    out = df.apply(lambda r: pd.Series(nearest_dist(r['LATITUDE'], r['LONGITUDE'], shopping_malls_xy),
                    index=['DIST_TO_NEAREST_SHOP_M']), axis=1) # in meters
    df = pd.concat([df, out], axis=1)

    return df


In [13]:
def preprocess(df_original, hdb_detail, mrt_stations, hawkers, shopping_malls):
    num_cols = ['FLOOR_AREA_SQM', 'LEASE_COMMENCE_DATA', 'LATITUDE', 'LONGITUDE']
    cat_cols = ['FLAT_TYPE']
    df = df_original.copy()
    df['STREET'] = df['STREET'].str.strip().str.lower()
    df = df.replace({
        '5-room': '5 room',
        '4-room': '4 room',
        '3-room': '3 room',
        '2-room': '2 room',
        '1-room': '1 room',
    })
    df = pd.get_dummies(df, columns=['FLAT_TYPE'])
    # TODO: floor range / flat model / lease commence data
    df['FLOOR_BUCKET'] = df['FLOOR_RANGE'].apply(classify_floor_range)
    df = pd.get_dummies(df, columns=['FLOOR_BUCKET'])

    df = add_additional_info(df, hdb_detail, mrt_stations, hawkers, shopping_malls)

    # drop unused data
    df = df.drop(['ECO_CATEGORY'], axis=1)
    # ['TOWN', 'BLOCK', 'STREET', 'FLAT_TYPE', 'FLOOR_BUCKET', 'FLOOR_RANGE']

    return df

In [14]:
df_cleaned = preprocess(df_dirty, hdb_detail, mrt_stations, hawkers, shopping_malls)

In [15]:
df_cleaned.shape

(162691, 24)

In [16]:
df_cleaned[['DIST_TO_NEAREST_MRT_M', 'DIST_TO_NEAREST_HAWKER_M', 'DIST_TO_NEAREST_SHOP_M']]

,DIST_TO_NEAREST_MRT_M,DIST_TO_NEAREST_HAWKER_M,DIST_TO_NEAREST_SHOP_M
0,306.013744,2683.955240,1994.701707
1,786.297948,916.936357,1176.138670
2,1052.540106,5364.143377,980.543397
3,872.296018,4563.650903,463.143097
4,1342.862156,2921.355463,851.672489
...,...,...,...
162686,627.321503,911.095177,828.902514
162687,759.006005,3608.220549,1670.498928
162688,443.281824,886.171063,2534.264925
162689,637.643152,4392.346046,594.746802
